# Logic Graph Examples

Create interactive graphs of logical formulas with inference edges.

**Smart Sampling:** Use the `max_nodes` parameter to intelligently limit graph size.
The sampler prioritizes: axioms → entailed formulas → tautologies → false statements (simpler formulas first).

For higher depths (3-4), try `max_nodes=200-300` for fast, focused visualizations!

In [1]:
# Example 1: Modus Ponens - x, (x → y) ⊢ y
from calculus_ratiocinator import Var, Implies, build_logic_graph, export_to_html

x = Var("x")
y = Var("y")
axioms = [x, Implies(x, y)]

# Build graph with smart sampling
g = build_logic_graph(["x", "y"], axioms, max_depth=2, max_nodes=300)

print(f"Graph has {g.number_of_nodes()} nodes and {g.number_of_edges()} edges")

# Check if y is entailed (it should be via modus ponens)
if str(y) in g.nodes:
    print(f"✓ y is entailed: {g.nodes[str(y)]['entailed']}")

# Export to interactive HTML (automatically saved in graphs/ folder)
export_to_html(g, "modus_ponens.html")

Graph has 300 nodes and 626 edges
✓ y is entailed: True
Interactive graph saved to graphs\modus_ponens.html


In [5]:
# Example 2: Multiple inference rules in one graph
from calculus_ratiocinator import Var, Implies, build_logic_graph, export_to_html

p = Var("p")
q = Var("q")
r = Var("r")

# Axioms: p, (p → q), (q → r)
axioms = [p, Implies(p, q), Implies(q, r)]

g = build_logic_graph(["p", "q", "r"], axioms, max_depth=2, max_nodes=200)

print(f"Graph has {g.number_of_nodes()} nodes and {g.number_of_edges()} edges")

# Check derived formulas
for formula_str in ["q", "r", "(p → r)"]:
    if formula_str in g.nodes:
        entailed = g.nodes[formula_str]['entailed']
        print(f"✓ {formula_str} entailed: {entailed}")

export_to_html(g, "complex.html")

Graph has 200 nodes and 290 edges
✓ q entailed: True
✓ r entailed: True
✓ (p → r) entailed: True
Interactive graph saved to graphs\complex.html


In [6]:
# Example 3: Explore formulas with specific axioms
from calculus_ratiocinator import Var, And, build_logic_graph, export_to_html

a = Var("a")
b = Var("b")

# Start with a conjunction axiom
axioms = [And(a, b)]

# Try depth 3 with smart sampling!
g = build_logic_graph(["a", "b"], axioms=axioms, max_depth=2, max_nodes=200)

print(f"Total formulas: {g.number_of_nodes()}")

# Count by category
axiom_count = sum(1 for _, d in g.nodes(data=True) if d.get('is_axiom'))
entailed_count = sum(1 for _, d in g.nodes(data=True) if d.get('entailed'))
tautologies = sum(1 for _, d in g.nodes(data=True) if d.get('tautology'))
print(f"Axioms: {axiom_count}")
print(f"Entailed formulas: {entailed_count}")
print(f"Tautologies: {tautologies}")

export_to_html(g, "conjunction.html")

Total formulas: 200
Axioms: 1
Entailed formulas: 200
Tautologies: 54
Interactive graph saved to graphs\conjunction.html


## Understanding the Graph

**Node colors:**
- 🟠 Orange: Axiom formulas (starting points)
- 🟢 Light Green: Tautologies (always true)
- 🔵 Sky Blue: Entailed from axioms
- 🔴 Red: Not entailed (false under axioms)

**Edge colors (inference rules):**
- Dark Gray-Blue: General entailment
- Hot Pink: Modus Ponens (MP)
- Deep Purple: Modus Tollens (MT)
- Amber: Disjunctive Syllogism (DS)
- Cyan: Hypothetical Syllogism (HS)
- Indigo: Conjunction Elimination Left (∧E-L)
- Brown: Conjunction Elimination Right (∧E-R)

**Interactive features:**
- Drag nodes to rearrange
- Hover over nodes/edges for details
- Scroll to zoom
- Use navigation buttons in bottom-right

**Smart Sampling:**
- Perfect for exploring depth 3-4 without overwhelming the browser!
- Set `max_nodes` to limit graph size
- Prioritizes: axioms → entailed → tautologies → false statements (simpler first)

# Classic Axiom Systems Collection

Below are several famous axiom systems you can experiment with!

In [2]:
# 1. Hilbert-style axioms for propositional logic
from calculus_ratiocinator import Var, Implies, Not, build_logic_graph, export_to_html

A = Var("A")
B = Var("B")
C = Var("C")

A1 = Implies(A, Implies(B, A))
A2 = Implies(Implies(A, Implies(B, C)), Implies(Implies(A, B), Implies(A, C)))
A3 = Implies(Implies(Not(A), Not(B)), Implies(B, A))
axioms = [A1, A2, A3]

# Need depth 3 to generate A2 and A3 (they have deeper nesting than A1)
g = build_logic_graph(["A", "B", "C"], axioms, max_depth=2, max_nodes=300)
print(f"Hilbert system: {g.number_of_nodes()} nodes, {g.number_of_edges()} edges")
export_to_html(g, "hilbert_axioms.html")

Hilbert system: 300 nodes, 287 edges
Interactive graph saved to graphs\hilbert_axioms.html


In [ ]:
# 2. Łukasiewicz's 3-axiom system (very elegant!)
from calculus_ratiocinator import Var, Implies, Not, build_logic_graph, export_to_html

A = Var("A")
B = Var("B")
C = Var("C")

# Hypothetical syllogism form
A1 = Implies(Implies(A, B), Implies(Implies(B, C), Implies(A, C)))
# Ex falso quodlibet
A2 = Implies(Not(A), Implies(A, B))
# Peirce's law
A3 = Implies(Implies(Implies(A, B), A), A)
axioms = [A1, A2, A3]

g = build_logic_graph(["A", "B", "C"], axioms, max_depth=2, max_nodes=300)
print(f"Łukasiewicz system: {g.number_of_nodes()} nodes, {g.number_of_edges()} edges")
export_to_html(g, "lukasiewicz_axioms.html")

In [ ]:
# 3. Simple conjunction axioms
from calculus_ratiocinator import Var, And, Implies, build_logic_graph, export_to_html

A = Var("A")
B = Var("B")

# Conjunction elimination left
A1 = Implies(And(A, B), A)
# Conjunction elimination right
A2 = Implies(And(A, B), B)
# Conjunction introduction
A3 = Implies(A, Implies(B, And(A, B)))
axioms = [A1, A2, A3]

g = build_logic_graph(["A", "B"], axioms, max_depth=2, max_nodes=300)
print(f"Conjunction system: {g.number_of_nodes()} nodes, {g.number_of_edges()} edges")
export_to_html(g, "conjunction_axioms.html")

In [ ]:
# 4. Disjunction introduction/elimination
from calculus_ratiocinator import Var, Or, Implies, Not, build_logic_graph, export_to_html

A = Var("A")
B = Var("B")

# Disjunction introduction left
A1 = Implies(A, Or(A, B))
# Disjunction introduction right
A2 = Implies(B, Or(A, B))
# Disjunctive syllogism
A3 = Implies(Or(A, B), Implies(Not(A), B))
axioms = [A1, A2, A3]

g = build_logic_graph(["A", "B"], axioms, max_depth=2, max_nodes=300)
print(f"Disjunction system: {g.number_of_nodes()} nodes, {g.number_of_edges()} edges")
export_to_html(g, "disjunction_axioms.html")

In [ ]:
# 5. Law of excluded middle + specific case
from calculus_ratiocinator import Var, Or, Implies, Not, build_logic_graph, export_to_html

A = Var("A")
B = Var("B")

# Law of excluded middle
A1 = Or(A, Not(A))
# A specific implication
A2 = Implies(A, B)
# A is true
A3 = A
axioms = [A1, A2, A3]

g = build_logic_graph(["A", "B"], axioms, max_depth=2, max_nodes=300)
print(f"Excluded middle system: {g.number_of_nodes()} nodes, {g.number_of_edges()} edges")
export_to_html(g, "excluded_middle_axioms.html")